In [1]:
import requests
from bs4 import BeautifulSoup
import os
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException
import math
import time
import pandas as pd

In [2]:
driver = webdriver.Firefox(executable_path=r'/Users/morganoneka/Documents/PersonalProjects/geckodriver')

## scraping the amazon search page

In [3]:
brand_page = 'https://www.amazon.com/s?k=As+I+Am&rh=n%3A3760911%2Cp_89%3AAs+I+Am&dc&qid=1650565270&rnid=2528832011&ref=sr_nr_p_89_1'
driver.get(brand_page)
time.sleep(5)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [7]:
links = soup.find_all("h2")

In [8]:
len(links)

48

## scraping weird display pages

In [9]:
brand_page = 'https://www.amazon.com/stores/page/1DB8A150-0F33-43F4-B681-6911B16BF1BD?ingress=2&visitId=3dec1212-38b5-45f5-b1df-26f94b3a7df5&ref_=ast_bln'
driver.get(brand_page)
time.sleep(5)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [11]:
links = soup.find_all("a", class_="ProductShowcase__title__3eXnB")
len(links)

2

## let's try to make a generalized item acquisition function

In [19]:
def get_item_list(brand_page):
    # pull up brand page
    driver.get(brand_page)
    time.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    item_listing = []
    
    # note that some pages have combinations of the different item listings so that's why this is 3 separate if statements not if/else
    # parse page like this: https://www.amazon.com/stores/page/AD6500C9-1CC9-47A6-AF55-30CAC721D436?ingress=2&visitId=6a918af0-3d48-4b0c-8963-93feb0ebbb20&ref_=ast_bln
    if len(soup.find_all("div", class_="columns")) > 0:
        items = soup.find_all("div", class_="columns")[0].find_all("li")
        for item in items:
            item_listing.append(get_item_url_from_listing(item))
    
    # parse page like this: https://www.amazon.com/s?k=As+I+Am&rh=n%3A3760911%2Cp_89%3AAs+I+Am&dc&qid=1650565270&rnid=2528832011&ref=sr_nr_p_89_1
    if len(soup.find_all("h2")) > 0:
        items = soup.find_all("h2")
        for item in items:
            item_listing.append(get_item_url_from_listing(item))
    
    # parse page like this: https://www.amazon.com/stores/page/1DB8A150-0F33-43F4-B681-6911B16BF1BD?ingress=2&visitId=3dec1212-38b5-45f5-b1df-26f94b3a7df5&ref_=ast_bln
    if len(soup.find_all("a", class_="ProductShowcase__title__3eXnB")) > 0:
        items = soup.find_all("a", class_="ProductShowcase__title__3eXnB")
        for item in items:
            item_listing.append(get_item_url_from_listing(item))
    
    
#     return(["https://www.amazon.com" + item.find_all("a")[0].get("href") for item in items])
    return(item_listing)

def get_item_url_from_listing(listing):
    try:
        if listing.name == 'a':
            return ("https://www.amazon.com" + listing.get("href") )
        else:
            return ("https://www.amazon.com" + listing.find_all("a")[0].get("href") )
    except:
        return ""

In [20]:
get_item_list('https://www.amazon.com/stores/page/1DB8A150-0F33-43F4-B681-6911B16BF1BD?ingress=2&visitId=3dec1212-38b5-45f5-b1df-26f94b3a7df5&ref_=ast_bln')

a
a


['https://www.amazon.com/Not-Your-Mothers-Cream-Styling/dp/B093466RZG?ref_=ast_sto_dp',
 'https://www.amazon.com/Not-Your-Mothers-Defining-2-Pack/dp/B09JWSFC86?ref_=ast_sto_dp']

## get images

In [42]:
url = "https://www.amazon.com/Products-Carols-Daughter-Enhancing-Moisture/dp/B07HYZCNKT?ref_=ast_sto_dp&th=1"

In [43]:
driver.get(url)



In [44]:
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [45]:
image = soup.find("img", id="landingImage")
image.get("src")

'https://m.media-amazon.com/images/I/81n1kQXecFL._SX679_.jpg'